In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load the dataset
df = pd.read_csv('D:\COURSES\AI\Epsilon Ai\Session 38\Fuel_Veciles\data\processed\datset_feul_cleaned1.csv')

# Create the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SUPERHERO])

# Define the layout of the app
app.layout = dbc.Container([
    dbc.Row(dbc.Col(html.H1("Vehicle Analysis Dashboard", className="text-center"), className="mb-4")),
    
    dbc.Row([
        dbc.Col([
            html.Label("Select Year Range:"),
            dcc.RangeSlider(
                id='year_slider',
                min=df['year'].min(),
                max=df['year'].max(),
                value=[df['year'].min(), df['year'].max()],
                marks={str(year): str(year) for year in range(df['year'].min(), df['year'].max()+1)},
                step=None
            )
        ], md=12, className="mb-4"),
    ]),
    
    dbc.Row([
        dbc.Col([
            html.Label("Select Fuel Type:"),
            dcc.Dropdown(
                id='fuel_type_dropdown',
                options=[{'label': ft, 'value': ft} for ft in df['fuel_type'].unique()],
                value=df['fuel_type'].unique().tolist(),
                multi=True
            )
        ], md=6, className="mb-4"),
        
        dbc.Col([
            html.Label("Select Vehicle Class:"),
            dcc.Dropdown(
                id='vehicle_class_dropdown',
                options=[{'label': vc, 'value': vc} for vc in df['class'].unique()],
                value=df['class'].unique().tolist(),
                multi=True
            )
        ], md=6, className="mb-4"),
    ]),
    
    dbc.Row([
        dbc.Col(dcc.Graph(id='fig_vehicles_by_year'), md=12, className="mb-4"),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='fig_avg_co2'), md=12, className="mb-4"),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='fig_avg_fuel_economy'), md=12, className="mb-4"),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='fig_top_vehicle_makes'), md=12, className="mb-4"),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='fig_fuel_types'), md=12, className="mb-4"),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='fig_vehicle_classes'), md=12, className="mb-4"),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='fig_top_10_brands'), md=12, className="mb-4"),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='fig_displacement_vs_mpg'), md=12, className="mb-4"),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='fig_drivetrain_trends'), md=12, className="mb-4"),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='fig_top_transmissions'), md=12, className="mb-4"),
    ]),
])

# Define callback to update graphs based on selected filters
@app.callback(
    [
        Output('fig_vehicles_by_year', 'figure'),
        Output('fig_avg_co2', 'figure'),
        Output('fig_avg_fuel_economy', 'figure'),
        Output('fig_top_vehicle_makes', 'figure'),
        Output('fig_fuel_types', 'figure'),
        Output('fig_vehicle_classes', 'figure'),
        Output('fig_top_10_brands', 'figure'),
        Output('fig_displacement_vs_mpg', 'figure'),
        Output('fig_drivetrain_trends', 'figure'),
        Output('fig_top_transmissions', 'figure'),
    ],
    [
        Input('year_slider', 'value'),
        Input('fuel_type_dropdown', 'value'),
        Input('vehicle_class_dropdown', 'value')
    ]
)
def update_graphs(selected_years, selected_fuel_types, selected_vehicle_classes):
    filtered_df = df[
        (df['year'] >= selected_years[0]) & 
        (df['year'] <= selected_years[1]) & 
        (df['fuel_type'].isin(selected_fuel_types)) & 
        (df['class'].isin(selected_vehicle_classes))
    ]
    
    fig_vehicles_by_year = create_figure_vehicles_by_year(filtered_df)
    fig_avg_co2 = create_figure_avg_co2(filtered_df)
    fig_avg_fuel_economy = create_figure_avg_fuel_economy(filtered_df)
    fig_top_vehicle_makes = create_figure_top_vehicle_makes(filtered_df)
    fig_fuel_types = create_figure_fuel_types(filtered_df)
    fig_vehicle_classes = create_figure_vehicle_classes(filtered_df)
    fig_top_10_brands = create_figure_top_10_brands(filtered_df)
    fig_displacement_vs_mpg = create_figure_displacement_vs_mpg(filtered_df)
    fig_drivetrain_trends = create_figure_drivetrain_trends(filtered_df)
    fig_top_transmissions = create_figure_top_transmissions(filtered_df)
    
    return fig_vehicles_by_year, fig_avg_co2, fig_avg_fuel_economy, fig_top_vehicle_makes, fig_fuel_types, fig_vehicle_classes, fig_top_10_brands, fig_displacement_vs_mpg, fig_drivetrain_trends, fig_top_transmissions

# Define the functions to create the figures
def create_figure_vehicles_by_year(df):
    fig = px.histogram(df, x='year', title='Distribution of Vehicles by Year', height=600)
    fig.update_layout(bargap=0.2)
    return fig

def create_figure_avg_co2(df):
    avg_co2_by_fuel_and_cylinders = df.groupby(['fuel_type', 'engine_cylinders'])['tailpipe_co2_in_grams_mile_ft1'].mean().reset_index()
    fig = px.bar(avg_co2_by_fuel_and_cylinders, x='fuel_type', y='tailpipe_co2_in_grams_mile_ft1', color='engine_cylinders',
                 barmode='group', title='Average CO2 Emissions by Fuel Type and Cylinder Count', height=600)
    return fig

def create_figure_avg_fuel_economy(df):
    avg_mpg_by_year = df.groupby('year')['composite_combined_mpg'].mean().reset_index()
    fig = px.line(avg_mpg_by_year, x='year', y='composite_combined_mpg', title='Average Fuel Economy Over Time', markers=True, height=600)
    return fig

def create_figure_top_vehicle_makes(df):
    top_makes = df['make'].value_counts().nlargest(20).reset_index()
    top_makes.columns = ['make', 'count']
    fig = px.bar(top_makes, x='make', y='count', title='Top 20 Vehicle Makes', height=600)
    return fig

def create_figure_fuel_types(df):
    fig = px.histogram(df, x='fuel_type', title='Distribution of Fuel Types', height=600)
    fig.update_layout(bargap=0.2)
    return fig

def create_figure_vehicle_classes(df):
    fig = px.histogram(df, x='class', title='Distribution of Vehicle Classes', height=600)
    fig.update_layout(bargap=0.2)
    return fig

def create_figure_top_10_brands(df):
    top_10_brands = df.groupby('make')['composite_combined_mpg'].mean().nlargest(10).reset_index()
    fig = px.bar(top_10_brands, x='make', y='composite_combined_mpg', title='Top 10 Car Brands Based on Fuel Economy', height=600)
    return fig

def create_figure_displacement_vs_mpg(df):
    fig = px.scatter(df, x='engine_displacement', y='composite_combined_mpg', title='Displacement vs Combined MPG', height=600)
    return fig

def create_figure_drivetrain_trends(df):
    drivetrain_trends = df.groupby(['year', 'drive']).size().unstack().reset_index()
    fig = px.line(drivetrain_trends, x='year', y=drivetrain_trends.columns[1:], title='Trends in Drivetrain Popularity Over Time', height=600)
    return fig

def create_figure_top_transmissions(df):
    top_transmissions = df.groupby('transmission')['composite_combined_mpg'].mean().nlargest(10).reset_index()
    fig = px.bar(top_transmissions, x='transmission', y='composite_combined_mpg', title='Top Ten Most Efficient Transmissions', height=600)
    return fig
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\Hamed\AppData\Local\Temp\ipykernel_15200\1997457755.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Hamed\AppData\Local\Temp\ipykernel_15200\1997457755.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
